In [2]:
import torch, dgl, accelerate
import torch.nn as nn
import torch.nn.functional as F
from accelerate.utils import set_seed
from dgl.distributed import partition_graph
import time
import numpy as np
import os
from tqdm import tqdm
import pandas as pd


In [69]:
# ## creating top 20 pseudo csv
# top50kg = pd.read_csv('psuedo_scores_top50.csv', low_memory=False)
# top50kg
# top20kg = top50kg.groupby(['y_idx', 'relation'], as_index=False, sort=False).apply(lambda x: x.head(20)).reset_index(drop=True)
# top20kg
top20kg.to_csv('psuedo_scores_top20.csv', index=False)

In [71]:
top20kg[top20kg.relation == 'contraindication']

,y_id,y_idx,x_id,x_idx,relation,score,y_type,x_type
341600,13924_12592_14672_13460_12591_12536_30861_8146...,2502.0,DB04200,2827.0,contraindication,2.217336,disease,drug
341601,13924_12592_14672_13460_12591_12536_30861_8146...,2502.0,DB01415,1348.0,contraindication,2.079678,disease,drug
341602,13924_12592_14672_13460_12591_12536_30861_8146...,2502.0,DB01150,1127.0,contraindication,2.005925,disease,drug
341603,13924_12592_14672_13460_12591_12536_30861_8146...,2502.0,DB01786,1571.0,contraindication,1.943809,disease,drug
341604,13924_12592_14672_13460_12591_12536_30861_8146...,2502.0,DB03225,2332.0,contraindication,1.943809,disease,drug
...,...,...,...,...,...,...,...,...
683195,400005.0,11204.0,DB01760,1554.0,contraindication,2.931828,disease,drug
683196,400005.0,11204.0,DB03087,2255.0,contraindication,2.931828,disease,drug
683197,400005.0,11204.0,DB03711,2581.0,contraindication,2.931828,disease,drug
683198,400005.0,11204.0,DB02924,2162.0,contraindication,2.931828,disease,drug


In [2]:
kg = pd.read_csv('data/kg.csv')
# kg = pd.read_csv('../../PrimeKG/datasets/data/kg/auxillary/data/kg_giant.csv')
kg

/tmp/ipykernel_7383/2413110704.py:2: DtypeWarning: Columns (2,6) have mixed types. Specify dtype option on import or set low_memory=False.
  kg = pd.read_csv('data/kg.csv')


,relation,display_relation,x_id,x_type,x_name,x_source,y_id,y_type,y_name,y_source
0,protein_protein,ppi,9796,gene/protein,PHYHIP,NCBI,56992,gene/protein,KIF15,NCBI
1,protein_protein,ppi,7918,gene/protein,GPANK1,NCBI,9240,gene/protein,PNMA1,NCBI
2,protein_protein,ppi,8233,gene/protein,ZRSR2,NCBI,23548,gene/protein,TTC33,NCBI
3,protein_protein,ppi,4899,gene/protein,NRF1,NCBI,11253,gene/protein,MAN1B1,NCBI
4,protein_protein,ppi,5297,gene/protein,PI4KA,NCBI,8601,gene/protein,RGS20,NCBI
...,...,...,...,...,...,...,...,...,...,...
8100493,anatomy_protein_absent,expression absent,4720,anatomy,cerebellar vermis,UBERON,140,gene/protein,ADORA3,NCBI
8100494,anatomy_protein_absent,expression absent,1377,anatomy,quadriceps femoris,UBERON,105378952,gene/protein,KLF18,NCBI
8100495,anatomy_protein_absent,expression absent,1379,anatomy,vastus lateralis,UBERON,105378952,gene/protein,KLF18,NCBI
8100496,anatomy_protein_absent,expression absent,2084,anatomy,heart left ventricle,UBERON,105378952,gene/protein,KLF18,NCBI


In [3]:
from txgnn import TxData, TxGNN, TxEval

saving_path = './saved_models/'
# split = 'random'
split = 'complex_disease'
# split = 'cell_proliferation'
# split = 'mental_health'
# split = 'cardiovascular'
# split = 'anemia'
# split = 'adrenal_gland'
print(split)

## self-supverised learning
# additional_train = [{'x_type':'gene/protein', 'x_id': '9796.0',	'relation':'protein_protein',	'y_type':'gene/protein',	'y_id':'56992.0',	'x_idx':27422.0,	'y_idx':19536.0,},
#          {'x_type':'gene/protein', 'x_id': '9796.0',	'relation':'protein_protein',	'y_type':'gene/protein',	'y_id':'56992.0',	'x_idx':27609.0,	'y_idx':19536.0,},
#          {'x_type':'gene/protein', 'x_id': '9796.0',	'relation':'protein_protein',	'y_type':'gene/protein',	'y_id':'56342.0',	'x_idx':27609.0,	'y_idx':19536.0,},
#          {'x_type':'gene/protein', 'x_id': '9796.0',	'relation':'protein_protein',	'y_type':'gene/protein',	'y_id':'24234.0',	'x_idx':24609.0,	'y_idx':22222.0,},
#          {'x_type':'gene/protein', 'x_id': '9796.0',	'relation':'protein_protein',	'y_type':'gene/protein',	'y_id':'324343.0',	'x_idx':11111.0,	'y_idx':19536.0,}]
# create_psuedo_edges = True

# # very_strt = time.time()
# TxData1 = TxData(data_folder_path = './data/')
# TxData1.prepare_split(split = split, seed = 1, no_kg = False, additional_train=additional_train, create_psuedo_edges=create_psuedo_edges)

complex_disease


In [4]:
# very_strt = time.time()
TxData1 = TxData(data_folder_path = './data/')
TxData1.prepare_split(split = split, seed = 1, no_kg = False, additional_train = pd.read_csv('psuedo_labels_15000.csv'))

TxGNN1 = TxGNN(
        data = TxData1, 
        weight_bias_track = False,
        proj_name = 'TxGNN',
        exp_name = 'TxGNN',
    )

# TxGNN1.model_initialize(n_hid = 100, 
#                       n_inp = 100, 
#                       n_out = 100, 
#                       proto = True,
#                       proto_num = 3,
#                       attention = False,
#                       sim_measure = 'all_nodes_profile',
#                       bert_measure = 'disease_name',
#                       agg_measure = 'rarity',
#                       num_walks = 200,
#                       walk_mode = 'bit',
#                       path_length = 2)

Found local copy...
Found local copy...
Found local copy...
Found saved processed KG... Loading...
Splits detected... Loading splits....
Creating DGL graph....
additional 
Done!


In [5]:
## extract all disease's ids
diseases1= kg[kg['x_type'] == 'disease']['x_id']
diseases2 = kg[kg['y_type'] == 'disease']['y_id']
disease_ids = pd.concat([diseases1, diseases2]).unique()
len(disease_ids)

## obtain all diseases' degree from disease-drug relation only
disease_drug1 = kg[(kg['x_type'] == 'disease') & (kg['y_type'] == 'drug')]['x_id']
disease_drug2 = kg[(kg['x_type'] == 'drug') & (kg['y_type'] == 'disease')]['y_id']
disease_drug_value_counts = pd.concat([disease_drug1, disease_drug2]).value_counts()
disease_drug_degree = disease_drug_value_counts.reindex(disease_ids).fillna(0).astype(int)
disease_drug_degree.sum()

## how many drugs are there?
# drug_ids_x = kg[kg['x_type'] == 'drug']['x_id']
# drug_ids_y = kg[kg['y_type'] == 'drug']['y_id']
# drug_ids_value_count = pd.concat([drug_ids_x, drug_ids_y]).value_counts()
# drug_ids_value_count

disease_drug_degree.index.values
low_disease = disease_drug_degree[disease_drug_degree < 1]

id_mapping = TxData1.retrieve_id_mapping()
id2idx = {id:idx for idx, id in id_mapping['idx2id_disease'].items()}
print(len(id2idx))
print(len(low_disease))
low_disease_idx = low_disease.index.map(lambda x: id2idx[x] if '_' in x else id2idx[x+'.0'])#.apply(lambda x: id2idx[x])
low_disease_idx = np.array(low_disease_idx)


17080
15026


In [6]:
TxGNN1.model_initialize(n_hid = 100, 
                      n_inp = 100, 
                      n_out = 100, 
                      proto = True,
                      proto_num = 3,
                      attention = False,
                      sim_measure = 'all_nodes_profile',
                      bert_measure = 'disease_name',
                      agg_measure = 'rarity',
                      num_walks = 200,
                      walk_mode = 'bit',
                      path_length = 2)
TxGNN1.load_pretrained('pre_trained_model_ckpt/2')

In [7]:
# data = [{'x_type':'gene/protein', 'x_id': '9796.0',	'relation':'protein_protein',	'y_type':'gene/protein',	'y_id':'56992.0',	'x_idx':27422.0,	'y_idx':19536.0,},
#          {'x_type':'gene/protein', 'x_id': '9796.0',	'relation':'protein_protein',	'y_type':'gene/protein',	'y_id':'56992.0',	'x_idx':34345.0,	'y_idx':19536.0,},
#          {'x_type':'gene/protein', 'x_id': '9796.0',	'relation':'protein_protein',	'y_type':'gene/protein',	'y_id':'56342.0',	'x_idx':27422.0,	'y_idx':19536.0,},
#          {'x_type':'gene/protein', 'x_id': '9796.0',	'relation':'protein_protein',	'y_type':'gene/protein',	'y_id':'24234.0',	'x_idx':33333.0,	'y_idx':22222.0,},
#          {'x_type':'gene/protein', 'x_id': '9796.0',	'relation':'protein_protein',	'y_type':'gene/protein',	'y_id':'324343.0',	'x_idx':11111.0,	'y_idx':19536.0,}]
# # TxData1.df_train.append(data1, ignore_index=True).drop_duplicates()
# TxData1.df_train[TxData1.df_train.append(data1, ignore_index=True).duplicated()]

In [8]:
from txgnn import TxEval

# disease_idxs = [7700.0, 8000.0]
# disease_idxs = [11764.0, 13924]
disease_idxs = low_disease_idx[:2]
TxEval = TxEval(model = TxGNN1)
result = TxEval.eval_disease_centric(disease_idxs = disease_idxs, 
                                     relation = 'indication',
                                     save_name = None, 
                                     return_raw="concise",
                                     save_result = False)
result

  0%|          | 0/2 [00:00<?, ?it/s]

{'dis_idx': {'11160_13119_13978_12060_12327_12670_13210_11067_12903_12293_12376_12375_11767_10965_12460_10967_11602_12002_11762_13386_14363_10933_12452_13365_13250_13826_12445_12326_11360_11392_13985_14739_11351_13489_12421_9076_13738_11279_14675_11286_13249_12485_10986_12420_14428_12170_12091_12442_11364_13984_12418_14237_13010_12355_912_14469_12273_13269_12602_11774_10807_12977_12003_12370_11192_10987_11991_12333_10860_13929_13471_11912_13537_13963_11799_13215_11553_14182_19588_14849': 1038.0,
  '8099_12497_12498': 15420.0},
 'ranked_drug_ids': {'11160_13119_13978_12060_12327_12670_13210_11067_12903_12293_12376_12375_11767_10965_12460_10967_11602_12002_11762_13386_14363_10933_12452_13365_13250_13826_12445_12326_11360_11392_13985_14739_11351_13489_12421_9076_13738_11279_14675_11286_13249_12485_10986_12420_14428_12170_12091_12442_11364_13984_12418_14237_13010_12355_912_14469_12273_13269_12602_11774_10807_12977_12003_12370_11192_10987_11991_12333_10860_13929_13471_11912_13537_13963_1179

In [9]:
## unit test
idx = np.where(np.array(result['ranked_drug_ids']['8099_12497_12498']) == 'DB00609')[0].item()
result['ranked_scores']['8099_12497_12498'][idx]
## 2.079557 is what we should get.

2.0795565

## Test environment for get_disease_score

In [10]:
df, df_train, df_valid, df_test, = TxData1.df, TxData1.df_train, TxData1.df_valid, TxData1.df_test, 
disease_rel_types = ['rev_contraindication', 'rev_indication', 'rev_off-label use']
device = torch.device("cuda")
model = TxGNN1.best_model.to(device)
G = TxGNN1.G.to(device)

def convert2str(x):
    try:
        if '_' in str(x): 
            pass
        else:
            x = float(x)
    except:
        pass

    return str(x)

df['x_id'] = df.x_id.apply(lambda x: convert2str(x))
df['y_id'] = df.y_id.apply(lambda x: convert2str(x))

idx2id_drug = dict(df[df.x_type == 'drug'][['x_idx', 'x_id']].drop_duplicates().values)
idx2id_drug.update(dict(df[df.y_type == 'drug'][['y_idx', 'y_id']].drop_duplicates().values))

idx2id_disease = dict(df[df.x_type == 'disease'][['x_idx', 'x_id']].drop_duplicates().values)
idx2id_disease.update(dict(df[df.y_type == 'disease'][['y_idx', 'y_id']].drop_duplicates().values))

data_path = './data/'
df_ = pd.read_csv(os.path.join(data_path, 'kg.csv'))
df_['x_id'] = df_.x_id.apply(lambda x: convert2str(x))
df_['y_id'] = df_.y_id.apply(lambda x: convert2str(x))

id2name_disease = dict(df_[df_.x_type == 'disease'][['x_id', 'x_name']].drop_duplicates().values)
id2name_disease.update(dict(df_[df_.y_type == 'disease'][['y_id', 'y_name']].drop_duplicates().values))

id2name_drug = dict(df_[df_.x_type == 'drug'][['x_id', 'x_name']].drop_duplicates().values)
id2name_drug.update(dict(df_[df_.y_type == 'drug'][['y_id', 'y_name']].drop_duplicates().values))

drug_ids_rels = {}
disease_ids_rels = {}

for i in ['indication', 'contraindication', 'off-label use']:
    drug_ids_rels['rev_' + i] = df[df.relation == i].x_id.unique()
    disease_ids_rels[i] = df[df.relation == i].y_id.unique()

In [16]:
def get_scores_disease(rel, disease_ids):
    df_train_valid = pd.concat([df_train, df_valid])
    df_dd = df_test[df_test.relation.isin(disease_rel_types)] ## filters df_test that has the relation of interest
    # df_dd_all = pd.concat([df_train, df_valid, df_test]) ## 
    df_dd_train = df_train_valid[df_train_valid.relation.isin(disease_rel_types)]

    df_rel_dd = df_dd[df_dd.relation == rel][['x_idx', 'y_idx']] ## shrink down df
    df_rel_dd_train = df_dd_train[df_dd_train.relation == rel][['x_idx', 'y_idx']] ## shrink down df
    drug_nodes = G.nodes('drug').cpu().numpy()
    if disease_ids is None:
        disease_ids = df_rel_dd.x_idx.unique()
    preds_contra = {}
    labels_contra = {}
    ids_contra = {}

    for disease_id in tqdm(disease_ids):

        candidate_pos = df_rel_dd[df_rel_dd.x_idx == disease_id]#[['x_idx', 'y_idx']]
        candidate_pos_train = df_rel_dd_train[df_rel_dd_train.x_idx == disease_id]
        drug_pos = candidate_pos.y_idx.values
        drug_pos_train_val = candidate_pos_train.y_idx.values

        labels = {}
        for i in drug_nodes:
            if i in drug_pos:
                labels[i] = 1
            elif i in drug_pos_train_val:
                labels[i] = -1
                # in the training set
            else:
                labels[i] = 0

        # construct eval graph
        out = {}
        src = torch.Tensor([disease_id] * len(labels)).to(device).to(dtype = torch.int64)
        dst = torch.Tensor(list(labels.keys())).to(device).to(dtype = torch.int64)
        out.update({('disease', rel, 'drug'): (src, dst)})
        print(out)
        g_eval = dgl.heterograph(out, num_nodes_dict={ntype: G.number_of_nodes(ntype) for ntype in G.ntypes}).to(device)
        
        model.eval()
        with torch.no_grad():
            _, pred_score_rel, _, _, _ = model(G, g_eval, psuedo=True)
        pred = pred_score_rel[('disease', rel, 'drug')].reshape(-1,).detach().cpu().numpy()
        lab = {idx2id_drug[i]: labels[i] for i in g_eval.edges()[1].detach().cpu().numpy()}
        preds_contra[idx2id_disease[disease_id]] = {idx2id_drug[i]: pred[idx] for idx, i in enumerate(g_eval.edges()[1].detach().cpu().numpy())}
        labels_contra[idx2id_disease[disease_id]] = lab
        # ids_contra[idx2id_disease[disease_id]] = g_eval.edges()[1].detach().cpu().numpy()

        del pred_score_rel
    return preds_contra, labels_contra, drug_nodes, [id2name_drug[idx2id_drug[i]] for i in drug_nodes]

In [19]:
def testing_scores(rel_type, disease_ids):
    temp_d, preds_all, labels_all, org_out_all, metrics_all, ranked_Ids, ranked_list, name, dis_id, dis_idx, ranked_Idxs, ranked_scores = {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}
    ## Q. rel_type = 'rev_' + relation ## why add rev_ ??
    preds_, labels_, drug_idxs, drug_names = get_scores_disease(rel_type, disease_ids)
    preds_all[rel_type] = preds_ ## Q. shouldn't the rel_type be in the dictionary as key then? A. No, because we are returning preds_all[relation] not preds_all
    labels_all[rel_type] = labels_
    ids_all = list(preds_all[rel_type].keys())
    id2idx_disease = {id:idx for idx, id in idx2id_disease.items()}
    for entity_id in ids_all:
        pred = preds_all[rel_type][entity_id]
        lab = labels_all[rel_type][entity_id]
        ids_rels = drug_ids_rels
        pred_array = np.array([v for k, v in pred.items()]) ## turns the dictionary into np.array, only extracting the scores
        id2idx = {i: idx for i, idx in enumerate(pred.keys())} ## takes the index of preds_ (int) and returns the drug id (str)
        ranked_list_entity = np.argsort(pred_array)[::-1] ## sort it in descending order
        ranked_scores[entity_id] = pred_array[ranked_list_entity]
        ranked_Idxs[entity_id] = ranked_list_entity

        ranked_Ids[entity_id] = [id2idx[i] for i in ranked_list_entity]
        # name[entity_id] = id2name_disease[entity_id]
        dis_idx[entity_id] = id2idx_disease[entity_id]

    out = {"dis_idx":dis_idx, "ranked_drug_ids": ranked_Ids, "ranked_drug_idxs": ranked_Idxs, "ranked_scores": ranked_scores}
    return out

In [20]:
rel_type = "rev_" + "indication"
results = testing_scores(rel_type, disease_idxs)
# preds_, labels_, drug_idxs, drug_names = get_scores_disease(rel_type, disease_idxs)

  0%|          | 0/2 [00:00<?, ?it/s]

{('disease', 'rev_indication', 'drug'): (tensor([1038, 1038, 1038,  ..., 1038, 1038, 1038], device='cuda:0'), tensor([   0,    1,    2,  ..., 7954, 7955, 7956], device='cuda:0'))}


 50%|█████     | 1/2 [00:00<00:00,  1.03it/s]

{('disease', 'rev_indication', 'drug'): (tensor([15420, 15420, 15420,  ..., 15420, 15420, 15420], device='cuda:0'), tensor([   0,    1,    2,  ..., 7954, 7955, 7956], device='cuda:0'))}


100%|██████████| 2/2 [00:01<00:00,  1.13it/s]


In [ ]:
## unit test
len(results['ranked_scores']['8099_12497_12498']) ## should see 7957


idx = np.where(np.array(results['ranked_drug_ids']['8099_12497_12498']) == 'DB00609')[0].item()
results['ranked_scores']['8099_12497_12498'][idx]
## 2.079557 is what we should get.

In [41]:
# import torch, dgl, accelerate
from txgnn import TxData, TxGNN, TxEval
# import torch.nn as nn
# import torch.nn.functional as F
# from accelerate.utils import set_seed
# from dgl.distributed import partition_graph
import numpy as np
import time
import pandas as pd
# saving_path = './saved_models/'
# split = 'random'
# split = 'complex_disease'
# split = 'cell_proliferation'
# split = 'mental_health'
# split = 'cardiovascular'
# split = 'anemia'
# split = 'adrenal_gland'
# print(split)

'''
    Let's first try one iteration to increase performance.
'''

def obtain_disease_idx(TxData1, deg):
    '''
        returns the disease idx that have less than k degrees (drug-disease relation)
    '''
    ## extract all disease's ids
    kg = pd.read_csv('data/kg.csv')
    diseases1= kg[kg['x_type'] == 'disease']['x_id']
    diseases2 = kg[kg['y_type'] == 'disease']['y_id']
    disease_ids = pd.concat([diseases1, diseases2]).unique()
    len(disease_ids)

    ## obtain all diseases' degree from disease-drug relation only
    disease_drug1 = kg[(kg['x_type'] == 'disease') & (kg['y_type'] == 'drug')]['x_id']
    disease_drug2 = kg[(kg['x_type'] == 'drug') & (kg['y_type'] == 'disease')]['y_id']
    disease_drug_value_counts = pd.concat([disease_drug1, disease_drug2]).value_counts()
    disease_drug_degree = disease_drug_value_counts.reindex(disease_ids).fillna(0).astype(int)
    disease_drug_degree.sum()

    ## length of ID
    # drug_ids_x = kg[kg['x_type'] == 'drug']['x_id']
    # drug_ids_y = kg[kg['y_type'] == 'drug']['y_id']
    # drug_ids_value_count = pd.concat([drug_ids_x, drug_ids_y]).value_counts()
    # drug_ids_value_count

    disease_drug_degree.index.values
    low_disease = disease_drug_degree[disease_drug_degree < deg]

    id_mapping = TxData1.retrieve_id_mapping()
    id2idx = {id:idx for idx, id in id_mapping['idx2id_disease'].items()}
    print(f"Total number of diseases?: {len(id2idx)}")
    print(f"total number of {deg}> degree diseases?: {len(low_disease)}")
    low_disease_idx = low_disease.index.map(lambda x: id2idx[x] if '_' in x else id2idx[x+'.0'])#.apply(lambda x: id2idx[x])
    low_disease_idx = np.array(low_disease_idx)

    return low_disease_idx[:2] ## testing

def turn_into_dataframe(results, t, least_score):
    '''
        t = number of psuedo_labels to be generated for low_diseases
        Takes in the results eval file and returns the disease idx that have less than k degrees (drug-disease relation)
    '''
    additional_train_dict = []
    for rel, result in results.items():
        for (dis_id, drug_ids), drug_idxs, dis_idx, ranked_scores in zip(result['ranked_drug_ids'].items(), result['ranked_drug_idxs'].values(), result['dis_idx'].values(), result['ranked_scores'].values()):
            if least_score is None:
                new_dicts = [{'y_id': dis_id, 'y_idx': dis_idx, 'x_id': drug_id, 'x_idx': drug_idx, 'relation': rel} for i, (drug_id, drug_idx) in enumerate(zip(drug_ids, drug_idxs)) if i < t]
            else:
                new_dicts = [{'y_id': dis_id, 'y_idx': dis_idx, 'x_id': drug_id, 'x_idx': drug_idx, 'relation': rel} for i, (drug_id, drug_idx, ranked_score) in enumerate(zip(drug_ids, drug_idxs, ranked_scores)) if ranked_score > least_score]
            additional_train_dict += new_dicts

    df = pd.DataFrame(additional_train_dict)
    df["x_idx"] = df["x_idx"].astype(float)
    df["y_type"] = "disease"
    df["x_type"] = "drug"
    return df

def generate_psuedo_labels(pre_trained_dir='pre_trained_model_ckpt/1', split='complex_disease', size=100, seed=1, deg=1, k_top_candidates=5, least_score=None):
    '''
        Loads a pre-trained model, calls (obtain_disease_idx, turn_into_dataframe) to generates psuedo_labels for diseases less than 'deg'. Returns dataframe ready to be augmented to df_train.
    '''
    strt = time.time()
    TxData1 = TxData(data_folder_path = './data/')
    TxData1.prepare_split(split=split, seed=seed, no_kg=False)
    low_disease_idx = obtain_disease_idx(TxData1=TxData1, deg=deg)

    txGNN = TxGNN(
                data = TxData1, 
                weight_bias_track = False,
                proj_name = 'TxGNN',
                exp_name = 'TxGNN'
            )
        
    txGNN.model_initialize(n_hid = size, 
                            n_inp = size, 
                            n_out = size, 
                            proto = True,
                            proto_num = 3,
                            attention = False,
                            sim_measure = 'all_nodes_profile',
                            bert_measure = 'disease_name',
                            agg_measure = 'rarity',
                            num_walks = 200,
                            walk_mode = 'bit',
                            path_length = 2)
    txGNN.load_pretrained(pre_trained_dir)
    disease_idxs = low_disease_idx
    txEval = TxEval(model = txGNN)
    indication = txEval.eval_disease_centric(disease_idxs = disease_idxs,
                                         relation = 'indication',
                                         save_name = None, 
                                         return_raw="concise",
                                         save_result = False)
    
    contraindication = txEval.eval_disease_centric(disease_idxs = disease_idxs, 
                                        relation = 'contraindication',
                                        save_name = None, 
                                        return_raw="concise",
                                        save_result = False)
    results =  {"indication":indication, "contraindication":contraindication}
    psuedo_training_df = turn_into_dataframe(results, t=k_top_candidates, least_score=least_score)
    psuedo_end = time.time() 
    print(f"time it took to generate psuedo_labels: {psuedo_end - strt}")
    return psuedo_training_df

def train_w_psuedo_labels(size=100, split='complex_disease', additional_train=None, create_psuedo_edges=False, seed=1, save_dir=None, dropout=0, reparam_mode=False, weight_decay=0):
    '''
        Takes in pretrained model and generate psuedo label? 
    '''
    strt = time.time()
    TxData1 = TxData(data_folder_path = './data/')
    ## add additional psuedo-training labels
    TxData1.prepare_split(split=split, seed=seed, no_kg=False, additional_train=additional_train, create_psuedo_edges=create_psuedo_edges,)
    TxGNN1 = TxGNN(
            data = TxData1, 
            weight_bias_track = False, #True,
            proj_name = 'TxGNN',
            exp_name = 'TxGNN'
        )
    TxGNN1.model_initialize(n_hid = size, 
                            n_inp = size, 
                            n_out = size, 
                            proto = True,
                            proto_num = 3,
                            attention = False,
                            sim_measure = 'all_nodes_profile',
                            bert_measure = 'disease_name',
                            agg_measure = 'rarity',
                            num_walks = 200,
                            walk_mode = 'bit',
                            path_length = 2,
                            dropout=dropout,
                            reparam_mode=reparam_mode)
    # Train
    TxGNN1.pretrain(n_epoch = 1, #---
                    learning_rate = 1e-3,
                    batch_size = 1024, 
                    train_print_per_n = 20)
    TxGNN1.finetune(n_epoch = 500, #---
                    learning_rate = 5e-4,
                    train_print_per_n = 5,
                    valid_per_n = 20,
                    weight_decay = weight_decay,)
    print(f"time it took for this training iteration: {time.time() - strt}")
    if save_dir is not None:
        noisy_student_fpath = './Noisy_student/'
        TxGNN1.save_model(path = noisy_student_fpath+save_dir)

In [42]:
import argparse
import os
import random

parser = argparse.ArgumentParser()
parser.add_argument('--dropout', default=0.0)
parser.add_argument('--reparam_mode', default=False, help='choose from {MLP, RMLP, MPNN}')
parser.add_argument('--psuedo_label_fname', default=None, help='choose from {psuedo_labels_75000.csv, }') ## is default None? 
parser.add_argument('--split', default='complex_disease', help='choose from {complex_disease, ...}')
parser.add_argument('--weight_decay', default=0.0, type=float)
parser.add_argument('--k_top_candidates', default=5, type=int)
parser.add_argument('--psuedo_edges', action='store_true')
parser.add_argument('--train_from_scratch', action='store_true')
parser.add_argument('--student_size', default=120, type=int)
parser.add_argument('--three_iter_from_scratch', action='store_true')
parser.add_argument('--use_diff_savedir', action='store_true')
parser.add_argument('--scaling_psuedo', action='store_true')
parser.add_argument('--testing', action='store_true')
parser.add_argument('--deg', default=1, type=float) ## 'inf' for all diseases?
parser.add_argument('--more_than_one_model_per_script', action='store_true')
parser.add_argument('--least_score', default=None, type=float)

## pre_trained_model
args, _ = parser.parse_known_args()
args.psuedo_label_fname = 'pl_least_score_5.csv'
args.least_score = 5.0

save_dir = 'Noisy_student2' if args.use_diff_savedir else 'Noisy_student'

seed = 1
if args.three_iter_from_scratch:
    # seed = random.randint(2, 100)
    print(f"Using seed{seed} to do full three iteration training")
    train_w_psuedo_labels(split=args.split, seed=seed, save_dir="Teacher")
    for i in range(3):
        print(f"generating and training Student{i+1}")
        args.k_top_candidates = args.k_top_candidates * (i+1) if args.scaling_psuedo else args.k_top_candidates ## scale num of psuedo labels
        size = 100 if i == 0 else args.student_size
        pre_trained_dir = f'./{save_dir}/Student{i}' if i > 0 else f"./{save_dir}/Teacher"
        psuedo_labels = generate_psuedo_labels(pre_trained_dir=pre_trained_dir, split=args.split, size=size, seed=seed, deg=args.deg, k_top_candidates=args.k_top_candidates)
        train_w_psuedo_labels(size=args.student_size, 
                        dropout=args.dropout, 
                        save_dir=f"Student{i+1}/", 
                        additional_train=psuedo_labels, 
                        create_psuedo_edges=args.psuedo_edges, 
                        split=args.split, 
                        reparam_mode=args.reparam_mode,
                        weight_decay=args.weight_decay)
# elif args.testing:
#     for i in range(2):
#         train_w_psuedo_labels(size=args.student_size, 
#                         dropout=args.dropout, 
#                         save_dir="The_First_Student/", 
#                         additional_train=None, 
#                         create_psuedo_edges=args.psuedo_edges, 
#                         split=args.split, 
#                         seed=seed,
#                         reparam_mode=args.reparam_mode,
#                         weight_decay=args.weight_decay)
elif args.train_from_scratch: ## To evaluate model at different seed
        ## should not be used until fixes
        seed = random.randint(2, 100)
        print(f"randomly picked seed is: {seed}")
        train_w_psuedo_labels(split=args.split, seed=seed, save_dir="random_seed")
        psuedo_labels = generate_psuedo_labels(pre_trained_dir=f'./{save_dir}/random_seed', split=args.split, size=100, seed=seed, deg=args.deg, k_top_candidates=args.k_top_candidates)
elif args.psuedo_label_fname is not None and os.path.exists(args.psuedo_label_fname):
    psuedo_labels = pd.read_csv(args.psuedo_label_fname)
    print(f"Loading generated psuedo_labels with size: {len(psuedo_labels)}")
    train_w_psuedo_labels(size=args.student_size, 
                        dropout=args.dropout, 
                        save_dir="The_First_Student/", 
                        additional_train=psuedo_labels, 
                        create_psuedo_edges=args.psuedo_edges, 
                        split=args.split, 
                        seed=seed,
                        reparam_mode=args.reparam_mode,
                            weight_decay=args.weight_decay)
else:
    print(f"Only generating {args.psuedo_label_fname}")
    psuedo_labels = generate_psuedo_labels(pre_trained_dir='pre_trained_model_ckpt/1', split=args.split, size=100, seed=seed, deg=args.deg, k_top_candidates=args.k_top_candidates, least_score=args.least_score)
    psuedo_labels.to_csv(args.psuedo_label_fname, index=False)

Only generating pl_least_score_5.csv
Found local copy...
Found local copy...
Found local copy...
Found saved processed KG... Loading...
Splits detected... Loading splits....
Creating DGL graph....
additional 
Done!
Total number of diseases?: 17080
total number of 1> degree diseases?: 15026


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

time it took to generate psuedo_labels: 173.7725486755371


In [43]:
psuedo_labels

,y_id,y_idx,x_id,x_idx,relation,y_type,x_type
0,11160_13119_13978_12060_12327_12670_13210_1106...,1038.0,DB09144,5823.0,indication,disease,drug
1,8099_12497_12498,15420.0,DB09357,5994.0,contraindication,disease,drug
2,8099_12497_12498,15420.0,DB09031,5716.0,contraindication,disease,drug
